1) EBNF + checagem de precedência (código)

In [ ]:
from textwrap import dedent

EBNF = dedent(r"""
program   := stmt* EOF ;
stmt      := 'let' ID '=' expr ';'
           | ID '=' expr ';'
           | 'print' expr ';'
           | 'if' '(' expr ')' block ('else' block)?
           | 'while' '(' expr ')' block
           | block ;
block     := '{' stmt* '}' ;

expr      := or ;
or        := and ( '||' and )* ;
and       := equality ( '&&' equality )* ;
equality  := relation ( ('==' | '!=') relation )* ;
relation  := add ( ('<' | '<=' | '>' | '>=') add )* ;
add       := mul ( ('+' | '-') mul )* ;
mul       := unary ( ('*' | '/') unary )* ;
unary     := ('-' | '!') unary | primary ;
primary   := NUMBER | ID | '(' expr ')' ;
""").strip()

print("=== EBNF ===\n")
print(EBNF)

# Ordem do mais forte -> mais fraco
prec_order = ["*","/","+","-","<","<=",">",">=","==","!=","&&","||"]

def check_precedence(order):
    idx = {op:i for i,op in enumerate(order)}
    assert idx["*"] < idx["+"] < idx["=="] < idx["&&"] < idx["||"]
    assert idx["/"] < idx["-"] < idx["<"]  < idx["&&"] < idx["||"]
    print("\nChecagem de precedência: OK")

check_precedence(prec_order)

=== EBNF ===

program   := stmt* EOF ;
stmt      := 'let' ID '=' expr ';'
           | ID '=' expr ';'
           | 'print' expr ';'
           | 'if' '(' expr ')' block ('else' block)?
           | 'while' '(' expr ')' block
           | block ;
block     := '{' stmt* '}' ;

expr      := or ;
or        := and ( '||' and )* ;
and       := equality ( '&&' equality )* ;
equality  := relation ( ('==' | '!=') relation )* ;
relation  := add ( ('<' | '<=' | '>' | '>=') add )* ;
add       := mul ( ('+' | '-') mul )* ;
mul       := unary ( ('*' | '/') unary )* ;
unary     := ('-' | '!') unary | primary ;
primary   := NUMBER | ID | '(' expr ')' ;

Checagem de precedência: OK


2) Lexer (código)

In [ ]:
import re
from dataclasses import dataclass
from typing import List

SPEC = [
    ("NUMBER", r"\d+"),
    ("KW", r"\b(let|print|if|else|while|true|false)\b"),
    ("ID", r"[A-Za-z_]\w*"),
    ("OP", r"==|!=|<=|>=|\|\||&&|[+\-*/=!<>]"),
    ("LP", r"\("), ("RP", r"\)"),
    ("LB", r"\{"), ("RB", r"\}"),
    ("SC", r";"),
    ("WS", r"[ \t\r\n]+"),
    ("COM", r"//[^\n]*"),
]
MASTER = re.compile("|".join(f"(?P<{n}>{p})" for n,p in SPEC))

@dataclass
class Tok:
    t: str; v: str; line: int; col: int

def lex(src:str)->List[Tok]:
    line=1; base=0; out=[]
    for m in MASTER.finditer(src):
        k=m.lastgroup; v=m.group(); col=m.start()-base+1
        if k in ("WS","COM"):
            for i,ch in enumerate(v):
                if ch=="\n": line+=1; base=m.start()+i+1
            continue
        out.append(Tok(k,v,line,col))
        if "\n" in v:
            for i,ch in enumerate(v):
                if ch=="\n": line+=1; base=m.start()+i+1
    out.append(Tok("EOF","",line,1)); return out

print("Lexer pronto.")


Lexer pronto.


3) Parser (descida recursiva)

In [ ]:
from dataclasses import dataclass
from typing import List, Optional

# AST
@dataclass
class Node: line:int; col:int
@dataclass
class Program(Node): body:List['Stmt']
@dataclass
class Stmt(Node): pass
@dataclass
class Block(Stmt): body:List['Stmt']
@dataclass
class Let(Stmt): name:str; expr:'Expr'
@dataclass
class Assign(Stmt): name:str; expr:'Expr'
@dataclass
class Print(Stmt): expr:'Expr'
@dataclass
class If(Stmt): cond:'Expr'; then:Block; els:Optional[Block]
@dataclass
class While(Stmt): cond:'Expr'; body:Block
@dataclass
class Expr(Node): pass
@dataclass
class Num(Expr): val:int
@dataclass
class Bool(Expr): val:int
@dataclass
class Var(Expr): name:str
@dataclass
class Unary(Expr): op:str; e:Expr
@dataclass
class Bin(Expr): op:str; l:Expr; r:Expr

class Parser:
    def __init__(self,toks:List[Tok]): self.t=toks; self.i=0
    def cur(self): return self.t[self.i]
    def eat(self,k, v=None):
        tok=self.cur()
        if tok.t==k and (v is None or tok.v==v): self.i+=1; return tok
        exp = f"{k}" + (f" '{v}'" if v else "")
        raise SyntaxError(f"Esperado {exp}, obtido {tok.t} '{tok.v}' @ {tok.line}:{tok.col}")
    def parse(self)->Program:
        body=[]
        while self.cur().t!="EOF": body.append(self.stmt())
        return Program(1,1,body)
    def block(self)->Block:
        l=self.eat("LB"); body=[]
        while self.cur().t!="RB": body.append(self.stmt())
        self.eat("RB"); return Block(l.line,l.col,body)
    def stmt(self)->Stmt:
        t=self.cur()
        if t.t=="KW" and t.v=="let":
            k=self.eat("KW","let"); name=self.eat("ID"); self.eat("OP","="); e=self.expr(); self.eat("SC")
            return Let(k.line,k.col,name.v,e)
        if t.t=="ID" and self.t[self.i+1].t=="OP" and self.t[self.i+1].v=="=":
            name=self.eat("ID").v; self.eat("OP","="); e=self.expr(); self.eat("SC")
            return Assign(t.line,t.col,name,e)
        if t.t=="KW" and t.v=="print":
            k=self.eat("KW","print"); e=self.expr(); self.eat("SC"); return Print(k.line,k.col,e)
        if t.t=="KW" and t.v=="if":
            k=self.eat("KW","if"); self.eat("LP"); c=self.expr(); self.eat("RP"); th=self.block(); el=None
            if self.cur().t=="KW" and self.cur().v=="else": self.eat("KW","else"); el=self.block()
            return If(k.line,k.col,c,th,el)
        if t.t=="KW" and t.v=="while":
            k=self.eat("KW","while"); self.eat("LP"); c=self.expr(); self.eat("RP"); b=self.block()
            return While(k.line,k.col,c,b)
        if t.t=="LB": return self.block()
        raise SyntaxError(f"Comando inválido @ {t.line}:{t.col}")
    # precedência
    def expr(self): return self.or_()
    def or_(self):
        e=self.and_()
        while self.cur().t=="OP" and self.cur().v=="||":
            op=self.eat("OP","||"); e=Bin(op.line,op.col,"||",e,self.and_())
        return e
    def and_(self):
        e=self.eq()
        while self.cur().t=="OP" and self.cur().v=="&&":
            op=self.eat("OP","&&"); e=Bin(op.line,op.col,"&&",e,self.eq())
        return e
    def eq(self):
        e=self.rel()
        while self.cur().t=="OP" and self.cur().v in ("==","!="):
            op=self.eat("OP",self.cur().v); e=Bin(op.line,op.col,op.v,e,self.rel())
        return e
    def rel(self):
        e=self.add()
        while self.cur().t=="OP" and self.cur().v in ("<","<=",">",">="):
            op=self.eat("OP",self.cur().v); e=Bin(op.line,op.col,op.v,e,self.add())
        return e
    def add(self):
        e=self.mul()
        while self.cur().t=="OP" and self.cur().v in ("+","-"):
            op=self.eat("OP",self.cur().v); e=Bin(op.line,op.col,op.v,e,self.mul())
        return e
    def mul(self):
        e=self.unary()
        while self.cur().t=="OP" and self.cur().v in ("*","/"):
            op=self.eat("OP",self.cur().v); e=Bin(op.line,op.col,op.v,e,self.unary())
        return e
    def unary(self):
        if self.cur().t=="OP" and self.cur().v in ("-","!"):
            op=self.eat("OP",self.cur().v); r=self.unary()
            return Unary(op.line, op.col, op.v, r)
        return self.primary()
    def primary(self):
        t=self.cur()
        if t.t=="NUMBER": self.eat("NUMBER"); return Num(t.line,t.col,int(t.v))
        if t.t=="KW" and t.v in ("true","false"):
            self.eat("KW",t.v); return Bool(t.line,t.col, 1 if t.v=="true" else 0)
        if t.t=="ID": self.eat("ID"); return Var(t.line,t.col,t.v)
        if t.t=="LP": self.eat("LP"); e=self.expr(); self.eat("RP"); return e
        raise SyntaxError(f"Expressão inválida @ {t.line}:{t.col}")

print("Parser pronto.")


Parser pronto.


4) Interpretador (escopos em pilha)

In [ ]:
class Env:
    def __init__(self): self.stack=[{}]
    def push(self): self.stack.append({})
    def pop(self): self.stack.pop()
    def declare(self, name, val): self.stack[-1][name]=val
    def set(self, name, val):
        for scope in reversed(self.stack):
            if name in scope:
                scope[name]=val; return
        raise RuntimeError(f"variável '{name}' não definida")
    def get(self, name):
        for scope in reversed(self.stack):
            if name in scope:
                return scope[name]
        raise RuntimeError(f"variável '{name}' não definida")

class Interp:
    def __init__(self): self.env = Env()
    def run(self, prog:Program):
        for st in prog.body: self.exec_stmt(st)
    def exec_stmt(self, s:Stmt):
        if isinstance(s, Let): self.env.declare(s.name, self.eval(s.expr)); return
        if isinstance(s, Assign): self.env.set(s.name, self.eval(s.expr)); return
        if isinstance(s, Print): print(self.eval(s.expr)); return
        if isinstance(s, Block):
            self.env.push()
            try:
                for x in s.body: self.exec_stmt(x)
            finally:
                self.env.pop()
            return
        if isinstance(s, If):
            if self.eval(s.cond): self.exec_stmt(s.then)
            elif s.els: self.exec_stmt(s.els)
            return
        if isinstance(s, While):
            while self.eval(s.cond): self.exec_stmt(s.body)
            return
        raise RuntimeError("Stmt não reconhecido")
    def eval(self, e:Expr):
        if isinstance(e, Num): return e.val
        if isinstance(e, Bool): return e.val
        if isinstance(e, Var):  return self.env.get(e.name)
        if isinstance(e, Unary):
            v=self.eval(e.e); return -v if e.op=="-" else (0 if v else 1)
        if isinstance(e, Bin):
            if e.op=="||":
                a=self.eval(e.l); return 1 if (a or self.eval(e.r)) else 0
            if e.op=="&&":
                a=self.eval(e.l); return 1 if (a and self.eval(e.r)) else 0
            a=self.eval(e.l); b=self.eval(e.r)
            if e.op=="+": return a+b
            if e.op=="-": return a-b
            if e.op=="*": return a*b
            if e.op=="/":
                if b==0: raise RuntimeError("divisão por zero")
                return a//b
            if e.op=="==": return 1 if a==b else 0
            if e.op=="!=": return 1 if a!=b else 0
            if e.op=="<":  return 1 if a<b else 0
            if e.op=="<=": return 1 if a<=b else 0
            if e.op==">":  return 1 if a>b else 0
            if e.op==">=": return 1 if a>=b else 0
        raise RuntimeError("Expr não reconhecida")

print("Interpretador pronto.")

Interpretador pronto.


5) Execução de exemplos + verificação (PASS/FAIL)

In [ ]:
import io, contextlib

def run_and_capture(src:str):
    prog = Parser(lex(src)).parse()
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
        Interp().run(prog)
    return buf.getvalue().strip().splitlines() if buf.getvalue() else []

src1 = """
let i=0;
while (i < 3) { print i; i = i + 1; }
if (i == 3) { print 100; } else { print 0; }
"""
out1 = run_and_capture(src1)
exp1 = ["0","1","2","100"]
print("[Programa 1] Saída:", out1, "| Esperado:", exp1, "|", "PASS" if out1==exp1 else "FAIL")

src2 = """
let x=1;
{ let x=5; print x; }
print x;
"""
out2 = run_and_capture(src2)
exp2 = ["5","1"]
print("[Programa 2] Saída:", out2, "| Esperado:", exp2, "|", "PASS" if out2==exp2 else "FAIL")

[Programa 1] Saída: ['0', '1', '2', '100'] | Esperado: ['0', '1', '2', '100'] | PASS
[Programa 2] Saída: ['5', '1'] | Esperado: ['5', '1'] | PASS


6) Projeto em disco: minic14/ + CLI + docs + testes

In [7]:
# Célula 6 (corrigida) — cria o projeto, garante import, roda CLI, testes e inclui depuração (tokens/AST)

import os, sys, shutil, subprocess, textwrap
from pathlib import Path

# --- 0) Preparação de diretórios ---
root = Path("minic14_project")
if root.exists():
    shutil.rmtree(root)
(root / "minic14").mkdir(parents=True)
(root / "examples").mkdir(parents=True)
(root / "tests").mkdir(parents=True)
(root / "docs").mkdir(parents=True)

# --- 1) Documentação básica ---
EBNF = textwrap.dedent(r"""
program   := stmt* EOF ;
stmt      := 'let' ID '=' expr ';'
           | ID '=' expr ';'
           | 'print' expr ';'
           | 'if' '(' expr ')' block ('else' block)?
           | 'while' '(' expr ')' block
           | block ;
block     := '{' stmt* '}' ;

expr      := or ;
or        := and ( '||' and )* ;
and       := equality ( '&&' equality )* ;
equality  := relation ( ('==' | '!=') relation )* ;
relation  := add ( ('<' | '<=' | '>' | '>=') add )* ;
add       := mul ( ('+' | '-') mul )* ;
mul       := unary ( ('*' | '/') unary )* ;
unary     := ('-' | '!') unary | primary ;
primary   := NUMBER | ID | '(' expr ')' ;
""").strip()

(root / "docs" / "EBNF.md").write_text(EBNF + "\n", encoding="utf-8")
(root / "README.md").write_text(textwrap.dedent("""
# minic14 — Projeto de referência (Módulo 14)
Uso:
  python -m minic14 examples/hello.minic
  python -m minic14 --dump-ast examples/loop.minic
"""), encoding="utf-8")

# --- 2) Código do pacote minic14/ ---
# 2.1 lexer.py
lexer_py = r'''
import re
from dataclasses import dataclass
from typing import List

SPEC = [
    ("NUMBER", r"\d+"),
    ("KW", r"\b(let|print|if|else|while|true|false)\b"),
    ("ID", r"[A-Za-z_]\w*"),
    ("OP", r"==|!=|<=|>=|\|\||&&|[+\-*/=!<>]"),
    ("LP", r"\("), ("RP", r"\)"),
    ("LB", r"\{"), ("RB", r"\}"),
    ("SC", r";"),
    ("WS", r"[ \t\r\n]+"),
    ("COM", r"//[^\n]*"),
]
MASTER = re.compile("|".join(f"(?P<{n}>{p})" for n,p in SPEC))

@dataclass
class Tok:
    t: str
    v: str
    line: int
    col: int

def lex(src: str) -> List[Tok]:
    line = 1; base = 0; out: List[Tok] = []
    for m in MASTER.finditer(src):
        k = m.lastgroup; v = m.group(); col = m.start() - base + 1
        if k in ("WS","COM"):
            for i,ch in enumerate(v):
                if ch == "\n": line += 1; base = m.start() + i + 1
            continue
        out.append(Tok(k, v, line, col))
        if "\n" in v:
            for i,ch in enumerate(v):
                if ch == "\n": line += 1; base = m.start() + i + 1
    out.append(Tok("EOF","",line,1))
    return out
'''
(root / "minic14" / "lexer.py").write_text(lexer_py, encoding="utf-8")

# 2.2 parser.py
parser_py = r'''
from dataclasses import dataclass
from typing import List, Optional
from .lexer import Tok, lex

@dataclass
class Node: line:int; col:int
@dataclass
class Program(Node): body:List['Stmt']
@dataclass
class Stmt(Node): pass
@dataclass
class Block(Stmt): body:List['Stmt']
@dataclass
class Let(Stmt): name:str; expr:'Expr'
@dataclass
class Assign(Stmt): name:str; expr:'Expr'
@dataclass
class Print(Stmt): expr:'Expr'
@dataclass
class If(Stmt): cond:'Expr'; then:Block; els:Optional[Block]
@dataclass
class While(Stmt): cond:'Expr'; body:Block
@dataclass
class Expr(Node): pass
@dataclass
class Num(Expr): val:int
@dataclass
class Bool(Expr): val:int
@dataclass
class Var(Expr): name:str
@dataclass
class Unary(Expr): op:str; e:Expr
@dataclass
class Bin(Expr): op:str; l:Expr; r:Expr

class Parser:
    def __init__(self,toks:List[Tok]): self.t=toks; self.i=0
    def cur(self): return self.t[self.i]
    def eat(self,k, v=None):
        tok=self.cur()
        if tok.t==k and (v is None or tok.v==v): self.i+=1; return tok
        exp = f"{k}" + (f" '{v}'" if v else "")
        raise SyntaxError(f"Esperado {exp}, obtido {tok.t} '{tok.v}' @ {tok.line}:{tok.col}")
    def parse(self)->Program:
        body=[]
        while self.cur().t!="EOF": body.append(self.stmt())
        return Program(1,1,body)
    def block(self)->Block:
        l=self.eat("LB"); body=[]
        while self.cur().t!="RB": body.append(self.stmt())
        self.eat("RB"); return Block(l.line,l.col,body)
    def stmt(self)->Stmt:
        t=self.cur()
        if t.t=="KW" and t.v=="let":
            k=self.eat("KW","let"); name=self.eat("ID"); self.eat("OP","="); e=self.expr(); self.eat("SC")
            return Let(k.line,k.col,name.v,e)
        if t.t=="ID" and self.t[self.i+1].t=="OP" and self.t[self.i+1].v=="=":
            name=self.eat("ID").v; self.eat("OP","="); e=self.expr(); self.eat("SC")
            return Assign(t.line,t.col,name,e)
        if t.t=="KW" and t.v=="print":
            k=self.eat("KW","print"); e=self.expr(); self.eat("SC"); return Print(k.line,k.col,e)
        if t.t=="KW" and t.v=="if":
            k=self.eat("KW","if"); self.eat("LP"); c=self.expr(); self.eat("RP"); th=self.block(); el=None
            if self.cur().t=="KW" and self.cur().v=="else": self.eat("KW","else"); el=self.block()
            return If(k.line,k.col,c,th,el)
        if t.t=="KW" and t.v=="while":
            k=self.eat("KW","while"); self.eat("LP"); c=self.expr(); self.eat("RP"); b=self.block()
            return While(k.line,k.col,c,b)
        if t.t=="LB": return self.block()
        raise SyntaxError(f"Comando inválido @ {t.line}:{t.col}")
    def expr(self): return self.or_()
    def or_(self):
        e=self.and_()
        while self.cur().t=="OP" and self.cur().v=="||":
            op=self.eat("OP","||"); e=Bin(op.line,op.col,"||",e,self.and_())
        return e
    def and_(self):
        e=self.eq()
        while self.cur().t=="OP" and self.cur().v=="&&":
            op=self.eat("OP","&&"); e=Bin(op.line,op.col,"&&",e,self.eq())
        return e
    def eq(self):
        e=self.rel()
        while self.cur().t=="OP" and self.cur().v in ("==","!="):
            op=self.eat("OP",self.cur().v); e=Bin(op.line,op.col,op.v,e,self.rel())
        return e
    def rel(self):
        e=self.add()
        while self.cur().t=="OP" and self.cur().v in ("<","<=",">",">="):
            op=self.eat("OP",self.cur().v); e=Bin(op.line,op.col,op.v,e,self.add())
        return e
    def add(self):
        e=self.mul()
        while self.cur().t=="OP" and self.cur().v in ("+","-"):
            op=self.eat("OP",self.cur().v); e=Bin(op.line,op.col,op.v,e,self.mul())
        return e
    def mul(self):
        e=self.unary()
        while self.cur().t=="OP" and self.cur().v in ("*","/"):
            op=self.eat("OP",self.cur().v); e=Bin(op.line,op.col,op.v,e,self.unary())
        return e
    def unary(self):
        if self.cur().t=="OP" and self.cur().v in ("-","!"):
            op=self.eat("OP",self.cur().v); r=self.unary()
            return Unary(op.line, op.col, op.v, r)
        return self.primary()
    def primary(self):
        t=self.cur()
        if t.t=="NUMBER": self.eat("NUMBER"); return Num(t.line,t.col,int(t.v))
        if t.t=="KW" and t.v in ("true","false"):
            self.eat("KW",t.v); return Bool(t.line,t.col, 1 if t.v=="true" else 0)
        if t.t=="ID": self.eat("ID"); return Var(t.line,t.col,t.v)
        if t.t=="LP": self.eat("LP"); e=self.expr(); self.eat("RP"); return e
        raise SyntaxError(f"Expressão inválida @ {t.line}:{t.col}")
'''
(root / "minic14" / "parser.py").write_text(parser_py, encoding="utf-8")

# 2.3 interp.py
interp_py = r'''
from .parser import Program, Stmt, Block, Let, Assign, Print, If, While, Expr, Num, Bool, Var, Unary, Bin

class Env:
    def __init__(self): self.stack=[{}]
    def push(self): self.stack.append({})
    def pop(self): self.stack.pop()
    def declare(self, name, val): self.stack[-1][name]=val
    def set(self, name, val):
        for scope in reversed(self.stack):
            if name in scope:
                scope[name]=val; return
        raise RuntimeError(f"variável '{name}' não definida")
    def get(self, name):
        for scope in reversed(self.stack):
            if name in scope:
                return scope[name]
        raise RuntimeError(f"variável '{name}' não definida")

class Interp:
    def __init__(self): self.env = Env()
    def run(self, prog:Program):
        for st in prog.body: self.exec_stmt(st)
    def exec_stmt(self, s:Stmt):
        if isinstance(s, Let): self.env.declare(s.name, self.eval(s.expr)); return
        if isinstance(s, Assign): self.env.set(s.name, self.eval(s.expr)); return
        if isinstance(s, Print): print(self.eval(s.expr)); return
        if isinstance(s, Block):
            self.env.push()
            try:
                for x in s.body: self.exec_stmt(x)
            finally:
                self.env.pop()
            return
        if isinstance(s, If):
            if self.eval(s.cond): self.exec_stmt(s.then)
            elif s.els: self.exec_stmt(s.els)
            return
        if isinstance(s, While):
            while self.eval(s.cond): self.exec_stmt(s.body)
            return
        raise RuntimeError("Stmt não reconhecido")
    def eval(self, e:Expr):
        if isinstance(e, Num): return e.val
        if isinstance(e, Bool): return e.val
        if isinstance(e, Var):  return self.env.get(e.name)
        if isinstance(e, Unary):
            v=self.eval(e.e); return -v if e.op=="-" else (0 if v else 1)
        if isinstance(e, Bin):
            if e.op=="||":
                a=self.eval(e.l); return 1 if (a or self.eval(e.r)) else 0
            if e.op=="&&":
                a=self.eval(e.l); return 1 if (a and self.eval(e.r)) else 0
            a=self.eval(e.l); b=self.eval(e.r)
            if e.op=="+": return a+b
            if e.op=="-": return a-b
            if e.op=="*": return a*b
            if e.op=="/":
                if b==0: raise RuntimeError("divisão por zero")
                return a//b
            if e.op=="==": return 1 if a==b else 0
            if e.op=="!=": return 1 if a!=b else 0
            if e.op=="<":  return 1 if a<b else 0
            if e.op=="<=": return 1 if a<=b else 0
            if e.op==">":  return 1 if a>b else 0
            if e.op==">=": return 1 if a>=b else 0
        raise RuntimeError("Expr não reconhecida")
'''
(root / "minic14" / "interp.py").write_text(interp_py, encoding="utf-8")

# 2.4 __main__.py (CLI)
main_py = r'''
import argparse, sys
from .lexer import lex
from .parser import Parser
from .interp import Interp

def main():
    ap = argparse.ArgumentParser(description="MiniC (subset) CLI")
    ap.add_argument("path", help="arquivo .minic")
    ap.add_argument("--dump-tokens", dest="dump_tokens", action="store_true")
    ap.add_argument("--dump-ast", action="store_true")
    args = ap.parse_args()

    with open(args.path, "r", encoding="utf-8") as f:
        src = f.read()

    toks = lex(src)
    if args.dump_tokens:
        for t in toks: print((t.t, t.v, t.line, t.col))
        return 0

    ast = Parser(toks).parse()
    if args.dump_ast:
        def pp(node, indent=0):
            pad="  "*indent
            cls=node.__class__.__name__
            fields={k:getattr(node,k) for k in getattr(node,"__dataclass_fields__",{}) if k not in ("line","col")}
            print(f"{pad}{cls} @{node.line}:{node.col} { {k:type(v).__name__ for k,v in fields.items()} }")
            for v in fields.values():
                if isinstance(v, list):
                    for x in v:
                        if hasattr(x, "__dataclass_fields__"): pp(x, indent+1)
                elif hasattr(v, "__dataclass_fields__"):
                    pp(v, indent+1)
        pp(ast); return 0

    Interp().run(ast); return 0

if __name__ == "__main__":
    sys.exit(main())
'''
(root / "minic14" / "__main__.py").write_text(main_py, encoding="utf-8")

# Torna o diretório um pacote (e os testes também, por robustez)
(root / "minic14" / "__init__.py").write_text("", encoding="utf-8")
(root / "tests" / "__init__.py").write_text("", encoding="utf-8")

# --- 3) Exemplos e testes ---
(root / "examples" / "hello.minic").write_text("print 123;\n", encoding="utf-8")
(root / "examples" / "loop.minic").write_text(
    "let i=0;\nwhile (i < 3) { print i; i = i + 1; }\nif (i == 3) { print 100; } else { print 0; }\n",
    encoding="utf-8"
)

tests_py = r'''
import io, contextlib
from minic14.lexer import lex
from minic14.parser import Parser
from minic14.interp import Interp

def run(src:str):
    prog = Parser(lex(src)).parse()
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
        Interp().run(prog)
    return buf.getvalue().strip().splitlines()

def test_loop():
    src = "let i=0;\nwhile (i < 3) { print i; i = i + 1; }\nif (i == 3) { print 100; } else { print 0; }\n"
    assert run(src) == ["0","1","2","100"]

def test_print():
    assert run("print 7;") == ["7"]
'''
(root / "tests" / "test_basic.py").write_text(tests_py, encoding="utf-8")

print("Estrutura criada em:", root.resolve())

# --- 4) Execução: define PYTHONPATH, roda CLI, AST e unittest ---
env = dict(os.environ)
env["PYTHONPATH"] = str(root.resolve())

print("\n== Demo CLI (hello.minic) ==")
res_cli = subprocess.run([sys.executable, "-m", "minic14", "examples/hello.minic"],
                         cwd=str(root), env=env, capture_output=True, text=True)
print("RC:", res_cli.returncode)
print("STDOUT:\n", res_cli.stdout if res_cli.stdout else "(vazio)")
print("STDERR:\n", res_cli.stderr if res_cli.stderr else "(vazio)")

print("\n== Dump AST (loop.minic) ==")
res_ast = subprocess.run([sys.executable, "-m", "minic14", "--dump-ast", "examples/loop.minic"],
                         cwd=str(root), env=env, capture_output=True, text=True)
print("RC:", res_ast.returncode)
print("STDOUT (primeiras 30 linhas):")
print("\n".join(res_ast.stdout.splitlines()[:30]) if res_ast.stdout else "(vazio)")
print("STDERR:\n", res_ast.stderr if res_ast.stderr else "(vazio)")

print("\n== Testes (unittest via módulo) ==")
res_tests = subprocess.run([sys.executable, "-m", "unittest", "-v", "tests.test_basic"],
                           cwd=str(root), env=env, capture_output=True, text=True)
print("RC:", res_tests.returncode)
print("STDOUT (últimas 60 linhas):")
print("\n".join(res_tests.stdout.splitlines()[-60:]) if res_tests.stdout else "(vazio)")
print("STDERR:\n", res_tests.stderr if res_tests.stderr else "(vazio)")

# --- 5) Depuração integrada (antiga célula 7): tokens e AST do pacote recém-criado ---
print("\n== Depuração: tokens e AST (via pacote) ==")
sys.path.insert(0, str(root.resolve()))
from minic14.lexer import lex as _lex
from minic14.parser import Parser as _Parser

code = "let x=2; if (x>1) { print x; } else { print 0; }"
print("Tokens:")
for t in _lex(code):
    print((t.t, t.v, t.line, t.col))

print("\nAST:")
ast = _Parser(_lex(code)).parse()
def _pp(node, indent=0):
    pad="  "*indent
    cls=node.__class__.__name__
    fields={k:getattr(node,k) for k in getattr(node,"__dataclass_fields__",{}) if k not in ("line","col")}
    print(f"{pad}{cls} @{node.line}:{node.col}")
    for v in fields.values():
        if isinstance(v, list):
            for x in v:
                if hasattr(x, "__dataclass_fields__"): _pp(x, indent+1)
        elif hasattr(v, "__dataclass_fields__"):
            _pp(v, indent+1)

_pp(ast)


Estrutura criada em: /content/minic14_project

== Demo CLI (hello.minic) ==
RC: 0
STDOUT:
 123

STDERR:
 (vazio)

== Dump AST (loop.minic) ==
RC: 0
STDOUT (primeiras 30 linhas):
Program @1:1 {'body': 'list'}
  Let @1:1 {'name': 'str', 'expr': 'Num'}
    Num @1:7 {'val': 'int'}
  While @2:1 {'cond': 'Bin', 'body': 'Block'}
    Bin @2:10 {'op': 'str', 'l': 'Var', 'r': 'Num'}
      Var @2:8 {'name': 'str'}
      Num @2:12 {'val': 'int'}
    Block @2:15 {'body': 'list'}
      Print @2:17 {'expr': 'Var'}
        Var @2:23 {'name': 'str'}
      Assign @2:26 {'name': 'str', 'expr': 'Bin'}
        Bin @2:32 {'op': 'str', 'l': 'Var', 'r': 'Num'}
          Var @2:30 {'name': 'str'}
          Num @2:34 {'val': 'int'}
  If @3:1 {'cond': 'Bin', 'then': 'Block', 'els': 'Block'}
    Bin @3:7 {'op': 'str', 'l': 'Var', 'r': 'Num'}
      Var @3:5 {'name': 'str'}
      Num @3:10 {'val': 'int'}
    Block @3:13 {'body': 'list'}
      Print @3:15 {'expr': 'Num'}
        Num @3:21 {'val': 'int'}
    Block @3